In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

laptimes_schema = StructType([
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), True),
    StructField("lap", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True)
])

In [0]:
laptimes_df = spark.read.schema(laptimes_schema).csv(f"{raw_folder_path}/{v_file_date}/lap_times")
display(laptimes_df)

In [0]:
from pyspark.sql.functions import current_timestamp, lit

laptimes_with_ingestion_date_df = laptimes_df.withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("raceId", "race_id") \
    .withColumn("file_date", lit(v_file_date))
laptimes_with_ingestion_date_df = add_ingestion_date(laptimes_with_ingestion_date_df)
display(laptimes_with_ingestion_date_df)

In [0]:
overwrite_partition(laptimes_with_ingestion_date_df, "f1_processed", "lap_times", "race_id")
display(spark.read.parquet(f"{processed_folder_path}/lap_times"))

In [0]:
dbutils.notebook.exit("Success")